In [7]:
!pip install transformers datasets huggingface_hub --quiet
!git clone https://github.com/l3cube-pune/indic-nlp.git


Cloning into 'indic-nlp'...
remote: Enumerating objects: 324, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 324 (delta 39), reused 43 (delta 4), pack-reused 213 (from 1)
Receiving objects: 100% (324/324), 1.46 GiB | 28.57 MiB/s, done.
Resolving deltas: 100% (100/100), done.
Updating files: 100% (147/147), done.


In [8]:
import os
os.listdir("indic-nlp/L3Cube-IndicQuest")


['Punjabi.csv',
 'Tamil.csv',
 'Konkani.csv',
 'Sindhi.csv',
 'Manipuri.csv',
 'Bengali.csv',
 'Marathi.csv',
 'eval_prompt.txt',
 'English.csv',
 'Maithili.csv',
 'Hindi.csv',
 'Odia.csv',
 'Gujarati.csv',
 'Nepali.csv',
 'Sanskrit.csv',
 'Assamese.csv',
 'Dogri.csv',
 'Telugu.csv',
 'README.md',
 'Malayalam.csv',
 'Urdu.csv',
 'Kannada.csv']

In [14]:
import pandas as pd

# Load language datasets
df_kn = pd.read_csv("indic-nlp/L3Cube-IndicQuest/Kannada.csv")
df_hi = pd.read_csv("indic-nlp/L3Cube-IndicQuest/Hindi.csv")
df_te = pd.read_csv("indic-nlp/L3Cube-IndicQuest/Telugu.csv")

# Preview
print(df_kn[['Question', 'Answer']].head())
print(df_hi[['Question', 'Answer']].head())
print(df_te[['Question', 'Answer']].head())


                                            Question  \
0   1920 ರಲ್ಲಿ ಅಸಹಕಾರ ಚಳುವಳಿಯ ಪ್ರಾರಂಭದೊಂದಿಗೆ ಯಾರ ...   
1   ಯಾವ ಭಾರತೀಯ ಕಾರ್ಯಕರ್ತನನ್ನು 'ಲೋಖಿತ್ವಾಡಿ' ಎಂದು ಜ...   
2   ಭಾರತದ ಮೊದಲ ಅಧ್ಯಕ್ಷರು ಮತ್ತು USA ನ ಮೊದಲ ಅಧ್ಯಕ್ಷ...   
3          ಅಬ್ದುಲ್ ಕಲಾಂ ಯಾರು? ಸಂಕ್ಷಿಪ್ತವಾಗಿ ವಿವರಿಸಿ.   
4   ಮೋದಿಯವರ ಮೊದಲ ಅವಧಿಯಲ್ಲಿ ಭಾರತದ ಹಣಕಾಸು ಮಂತ್ರಿ ಯಾರು?   

                                              Answer  
0   1 ಆಗಸ್ಟ್ 1920 ರಂದು, ಅಸಹಕಾರ ಚಳುವಳಿಯನ್ನು ಘೋಷಿಸಲ...  
1   ರಾವ್ ಬಹದ್ದೂರ್ ಗೋಪಾಲ್ ಹರಿ ದೇಶಮುಖ್ () ಅವರು ಲೋಖಿ...  
2   ರಾಜೇಂದ್ರ ಪ್ರಸಾದ್ ಅವರು ಭಾರತೀಯ ರಾಜಕಾರಣಿ, ವಕೀಲರು...  
3   ಎಪಿಜೆ ಅಬ್ದುಲ್ ಕಲಾಂ (ಜನನ ಅಕ್ಟೋಬರ್ 15, 1931, ರಾ...  
4   ಅರುಣ್ ಜೇಟ್ಲಿ (28 ಡಿಸೆಂಬರ್ 1952 - 24 ಆಗಸ್ಟ್ 20...  
                                            Question  \
0   1920 में असहयोग आंदोलन की शुरुआत के साथ ही कि...   
1   कौन से भारतीय कार्यकर्ता लोकप्रिय रूप से 'लोक...   
2   भारत के प्रथम राष्ट्रपति और संयुक्त राज्य अमे...   
3        अब्दुल कलाम कौन थे? संक्षेप में वर्णन करें।   
4   मोदी के पहले कार्यकाल के दौरान भारत के वित्त ... 

In [12]:
!pip install transformers gradio --quiet


In [17]:
import gradio as gr
import pandas as pd
from transformers import pipeline

# Load multilingual model
qa_pipeline = pipeline("question-answering", model="deepset/xlm-roberta-large-squad2")

# Load CSVs
df_kn = pd.read_csv("indic-nlp/L3Cube-IndicQuest/Kannada.csv")
df_hi = pd.read_csv("indic-nlp/L3Cube-IndicQuest/Hindi.csv")
df_te = pd.read_csv("indic-nlp/L3Cube-IndicQuest/Telugu.csv")

# Sample fallback context
default_contexts = {
    "Kannada": df_kn['Answer'][0],
    "Hindi": df_hi['Answer'][0],
    "Telugu": df_te['Answer'][0]
}

# Inference function
def answer_question(question, context, language):
    if not context.strip():
        context = default_contexts[language]
    try:
        result = qa_pipeline(question=question, context=context)
        return result["answer"]
    except Exception as e:
        return f"❌ Error: {str(e)}"

# Gradio UI: BhashaBot
with gr.Blocks(theme=gr.themes.Default(primary_hue="orange")) as iface:
    gr.Markdown("## 🤖 BhashaBot")
    gr.Markdown("India's multilingual Q&A assistant 🇮🇳 — Ask questions in **Kannada**, **Hindi**, or **Telugu**.")

    with gr.Row():
        lang = gr.Radio(choices=["Kannada", "Hindi", "Telugu"], label="🌐 Select Language", value="Kannada")

    question = gr.Textbox(label="❓ Enter Your Question", placeholder="Type your question here...")
    context = gr.Textbox(label="📜 Context (Leave blank to use default)", lines=4, placeholder="Paste passage or leave blank...")
    output = gr.Textbox(label="📥 Answer", lines=2)

    with gr.Row():
        submit_btn = gr.Button("🚀 Get Answer")
        clear_btn = gr.Button("🔁 Clear All")

    submit_btn.click(fn=answer_question, inputs=[question, context, lang], outputs=output)
    clear_btn.click(fn=lambda: ["", "", "Kannada", ""], inputs=[], outputs=[question, context, lang, output])

iface.launch()


Some weights of the model checkpoint at deepset/xlm-roberta-large-squad2 were not used when initializing XLMRobertaForQuestionAnswering: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a0b90d7e055ff90c88.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
